In [1]:
import pandas as pd # for handling csv and csv contents using data frames
import rdflib # for handling RDF(S) data, generating triples, posing sparql queries etc.
from SPARQLWrapper import SPARQLWrapper, JSON, RDFXML, TURTLE, CSV # for generating and executing SPARQL query over the SPARQL endpoint.
from rdflib.namespace import FOAF , XSD, RDF # most common namespaces (help us to type our data with some common knowledge)
import urllib.parse # for parsing strings to URI's
from rdflib import Graph, URIRef, Literal, BNode

In [7]:
authors_df = pd.read_csv('../Data/authors_final.csv')
genres_df = pd.read_csv('../Data/genres_pages_x_books.csv')
interactions_df = pd.read_csv('../Data/interactions_final.csv')
filtered_books_df = pd.read_csv('../Data/books_final.csv')

In [8]:
filtered_books_df

,book_id,similar_books,publication_year,title,writers,illustrators,contributors,editors,translators
0,977525,[],1999.0,A Boy in Winter,['73435'],[],[],[],[]
1,38569,[],2001.0,The Sound Bite Society: How Television Helps t...,['21706'],[],[],[],[]
2,38568,"['225669', '50789', '383143', '293616', '75960...",2005.0,A Quick Bite (Argeneau #1),['17059'],[],[],[],[]
3,38561,[],2006.0,Bite by Bite,['3370'],[],[],[],[]
4,38563,[],2004.0,Dogs Don't Bite When a Growl Will Do: What You...,"['21700', '21699']",[],[],[],[]
...,...,...,...,...,...,...,...,...,...
1238,763122,[],2007.0,"Hell Town (The Last Gunfighter, #16)",['20852'],[],[],[],[]
1239,763125,"['6596737', '1863391', '615157', '10635926', '...",2005.0,"The Drifter (The Last Gunfighter, #1)",['20852'],[],[],[],[]
1240,16195,"['6990584', '4788279', '62665', '129462', '153...",2003.0,Howard Zinn on War,['1899'],[],[],[],[]
1241,16196,"['624627', '12967503', '450577', '855492', '10...",2002.0,Emma,['1899'],[],[],[],[]


In [5]:
g = rdflib.Graph()
aut = rdflib.Namespace('http://my_project/author')
bk = rdflib.Namespace('http://my_project/book')
lec = rdflib.Namespace('http:my_project/lector')

In [ ]:
for index, row in authors_df.iterrows():
    g.add((rdflib.URIRef(aut+row['author_id']),RDF.type, ))
    g.add((rdflib.URIRef(ppl+row['Name']), RDF.type, FOAF.Person))
    g.add((rdflib.URIRef(ppl+row['Name']), rdflib.URIRef(schema+'name'), rdflib.Literal(row['Name'], datatype=XSD.string) ))
    g.add((rdflib.URIRef(ppl+row['Name']), FOAF.age, rdflib.Literal(row['Age'], datatype=XSD.integer) ))
    g.add((rdflib.URIRef(ppl+row['Name']), rdflib.URIRef(schema+'address'), rdflib.Literal(row['Address'], datatype=XSD.string) ))
    g.add((rdflib.URIRef(loc+urllib.parse.quote(row['Address'])), rdflib.URIRef(schema+'name'), rdflib.Literal(row['Address'], datatype=XSD.string) ))

In [11]:
# Loop through authors dataframe
for row in authors_df.itertuples():
    author_uri = BNode()
    g.add((author_uri, RDF.TYPE, aut['Author']))  # Use aut['Author'] for consistent namespace reference

    g.add((author_uri, aut['name'], Literal(row.name, datatype=XSD.string)))
    g.add((author_uri, aut['averageRating'], Literal(row.average_rating, datatype=XSD.float)))
    g.add((author_uri, aut['ratingsCount'], Literal(row.ratings_count, datatype=XSD.integer)))

# Loop through filtered_books dataframe
for row in filtered_books_df.itertuples():
    book_uri = BNode()
    g.add((book_uri, RDF.TYPE, bk['Book']))  # Use bk['Book'] for consistent namespace reference

    g.add((book_uri, bk['title'], Literal(row.title, datatype=XSD.string)))
    g.add((book_uri, bk['publicationYear'], Literal(row.publication_year, datatype=XSD.string)))

    for author_name in row.authors:
        author_uri = URIRef(f'{aut}{author_name}')  # Assuming author names are unique identifiers within your project
        g.add((book_uri, bk['writtenBy'], author_uri))

    for similar_book_title in row.similar_books:
        similar_book_uri = BNode()
        g.add((similar_book_uri, RDF.TYPE, bk['Book']))  # Use bk['Book'] for consistent namespace reference
        g.add((similar_book_uri, bk['title'], Literal(similar_book_title, datatype=XSD.string)))
        g.add((book_uri, bk['similarTo'], similar_book_uri))

# Loop through genres dataframe
for row in genres_df.itertuples():
    genre_uri = URIRef(f'http://my_project/genre/{row.name}')  # Assuming genre names are URLs within your project
    g.add((genre_uri, RDF.TYPE, bk['Genre']))  # Genre might be a sub-class of bk (modify if needed)

    g.add((genre_uri, bk['probability'], Literal(row[row.name], datatype=XSD.float)))  # Assuming probability is a column in genres_df

    book_uri = URIRef(f'{bk}Book/{row.book_id}')  # Assuming book_id is a URL within your project
    g.add((book_uri, bk['belongsTo'], genre_uri))

# Loop through interactions dataframe
for row in interactions_df.itertuples():
    user_uri = BNode()
    g.add((user_uri, RDF.TYPE, lec['User']))  # Use lec['User'] for consistent namespace reference
    g.add((user_uri, lec['id'], Literal(row.user_id, datatype=XSD.integer)))

    book_uri = URIRef(f'{bk}Book/{row.book_id}')  # Assuming book_id is a URL within your project
    g.add((user_uri, lec['read'], book_uri) if row.is_read else None)
    g.add((user_uri, lec['rated'], book_uri) if row.rating else None)
    g.add((user_uri, lec['reviewed'], book_uri) if row.is_reviewed else None)

AttributeError: term 'TYPE' not in namespace 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'

In [9]:
for a in authors_df.itertuples():
    print(a)
    break

Pandas(Index=0, average_rating=4.08, author_id=626222, name='Anita Diamant', ratings_count=546796)
